<a href="https://colab.research.google.com/github/WedyanFawaz/advanced_ai_exercises/blob/main/lab2_vanilla_gan_debugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Debug a Broken Vanilla GAN (find 12+ issues)

In [11]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torch

In [17]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5],[.5])])  # BUG normalize

In [18]:
loader=DataLoader(torchvision.datasets.MNIST('./data',download=True,train=True,transform=transform),batch_size=256,shuffle=True,num_workers=2,pin_memory=True)

In [78]:
len(loader.dataset[0][0])

1

In [57]:
z_dim=100; g_lr=2e-2; d_lr=2e-5
class D(nn.Module):
    def __init__(self):
        super().__init__();
        self.net=nn.Sequential(
            nn.Conv2d(1,32,4,2,1),
            nn.LeakyReLU(0.2,True),
            nn.Conv2d(32,64,4,2,1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2,True),
            nn.Conv2d(64,1,7,1,0),
            nn.Sigmoid())
    def forward(self,x):return self.net(x).view(x.size(0),1)


In [73]:
class G(nn.Module):
    def __init__(self):
        super().__init__();
        self.net=nn.Sequential(
            nn.ConvTranspose2d(100,128,7,1,0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1),
            nn.BatchNorm2d(64)
            ,nn.ReLU(True),
            nn.ConvTranspose2d(64,1,4,2,1),

            nn.Tanh())  # BUG missing Tanh
    def forward(self,z):
       z=z.view(z.size(0),100,1,1);
       return self.net(z)  # BUG reshape
Dnet=D().to(device); Gnet=G().to(device)
crit=nn.BCEWithLogitsLoss();
opt_d=torch.optim.Adam(Dnet.parameters(),lr=d_lr,betas=(0.9,0.999));
opt_g=torch.optim.Adam(Gnet.parameters(),lr=g_lr,betas=(0.9,0.999))

In [79]:
from tqdm import tqdm
for real,_ in tqdm(loader):
    real=real.to(device); b=real.size(0)
    # Train Discriminator
    opt_d.zero_grad()
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z)
    loss_d=crit(Dnet(real),torch.ones(b,1,device=device))+crit(Dnet(fake.detach()),torch.zeros(b,1,device=device)) # Corrected labels and detached fake
    loss_d.backward(); opt_d.step() # Stepping Discriminator optimizer

    # Train Generator
    opt_g.zero_grad()
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z)
    loss_g=crit(Dnet(fake),torch.ones(b,1,device=device)) # Corrected to use non-saturating loss
    loss_g.backward(); opt_g.step() # Stepping Generator optimizer

print('Now fix all the issues.')

  0%|          | 0/235 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 235/235 [03:27<00:00,  1.13it/s]

Now fix all the issues.
